In [1]:
import random
import time
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
from tqdm import tqdm
import gc

# Загрузка данных
data = pd.read_csv('houston_short_300.csv')
# data = data.sample(n=300, random_state=42)
# data.to_csv('houston_short_300.csv', index=False)
X = data[['Latitude', 'Longitude', 'Year Built', 'Beds', 'Baths', 'buildingSize', 'lotSize', 'PostalCode']]
y = data['Price']

In [2]:


# Функция оценки модели LGBM с использованием MSE
def lgbm_function(individual):
    params = {
        'n_estimators': int(individual[0]),
        'max_depth': int(individual[1]),
        'num_leaves': int(individual[2]),
        'learning_rate': individual[3],
        'max_bin': int(individual[4]),
        'colsample_bytree': individual[5]
    }
    
    model = lgb.LGBMRegressor(**params)
    scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_squared_error')
    mse = -np.mean(scores)
    return mse,

# Ограничение значений после мутации и кроссинговера
def check_bounds(individual, param_ranges):
    for i in range(len(individual)):
        individual[i] = max(param_ranges[f'x{i}'][0], min(individual[i], param_ranges[f'x{i}'][1]))
    return individual

# Настройка среды DEAP для генетического алгоритма
def genetic_algorithm_deap(param_ranges, population_size, generations, mutation_rate, crossover_rate,
                           elitism_fraction, alpha, mu, sigma, tournsize, crossover_method, mutation_method,
                           selection_method):
    # Создание минимизирующей функции в DEAP
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # Функция инициализации индивидов
    def create_individual():
        return creator.Individual([random.uniform(*param_ranges[param]) for param in param_ranges])

    # Настройка инструментария DEAP
    toolbox = base.Toolbox()
    toolbox.register("individual", create_individual)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", lgbm_function)

    # Handling crossover strategies
    toolbox.register("mate", crossover_method, alpha=alpha)
    toolbox.register("mutate", mutation_method, mu=mu, sigma=sigma, indpb=mutation_rate)
    toolbox.register("select", selection_method, tournsize=tournsize)

    # Инициализация популяции
    population = toolbox.population(n=population_size)

    # Добавление элитизма
    elite_size = int(population_size * elitism_fraction)

    # Старт отсчета времени
    start_time = time.time()

    best_individual = None
    best_score = float('inf')
    best_generation = 0
    times = []  # Сохраняем время каждой итерации

    # Запуск алгоритма
    for gen in range(generations):
        gen_start_time = time.time()
        offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_rate, mutpb=mutation_rate)

        # Применение функции check_bounds ко всем потомкам
        offspring = [check_bounds(ind, param_ranges) for ind in offspring]

        fits = list(map(toolbox.evaluate, offspring))

        # Оценка лучших индивидов
        for ind, fit in zip(offspring, fits):
            ind.fitness.values = fit
            if fit[0] < best_score:
                best_individual = ind
                best_score = fit[0]
                best_generation = gen + 1

        population = tools.selBest(offspring, elite_size) + tools.selBest(offspring, population_size - elite_size)
        
        # Сохраняем время итерации
        gen_time = time.time() - gen_start_time
        times.append(gen_time)

        print(f"Поколение {gen + 1}, Лучшее MSE: {best_score}, Параметры: {best_individual}")

    end_time = time.time()
    elapsed_time = end_time - start_time

    return best_individual, best_score, best_generation, elapsed_time, times

# Генерация случайных гиперпараметров
def generate_random_hyperparameters():
    hyperparams = {
        "population_size": random.randint(10, 100),
        "mutation_rate": random.uniform(0.01, 0.5),
        "crossover_rate": random.uniform(0.01, 0.5),
        "elitism_fraction": random.uniform(0.01, 0.2),
        "alpha": random.uniform(0.01, 0.9),  # For crossover methods like Blend
        "mu": random.uniform(-1, 1),  # For Gaussian mutation
        "sigma": random.uniform(0.01, 1),  # For Gaussian mutation
        "tournsize": random.randint(2, 5)
    }
    return hyperparams

# Сохранение результатов в CSV
def save_results_to_csv(df, filename):
    try:
        df.to_csv(filename, index=False, mode='a', header=not pd.io.common.file_exists(filename))
    except Exception as e:
        print(f"Ошибка сохранения в CSV: {e}")

# Внешний и внутренний циклы для сбора статистики
num_outer_runs = 2
num_inner_runs = 5
summary_results = []

param_ranges = {
    'x0': (50, 2000),        # n_estimators
    'x1': (1, 100),          # max_depth
    'x2': (2, 1000),         # num_leaves
    'x3': (0.01, 0.3),       # learning_rate
    'x4': (10, 1000),        # max_bin
    'x5': (0.5, 1.0)         # colsample_bytree
}

filename = "deap_houses.csv"  # Имя файла для сохранения результатов

# Внешний цикл
for outer_run in tqdm(range(num_outer_runs), desc="Внешние прогоны"):
    print(f"\n=== Внешний прогон {outer_run + 1} ===\n")
    
    hyperparams = generate_random_hyperparameters()
    results = []

    # Внутренний цикл
    for inner_run in range(num_inner_runs):
        best_solution, best_fitness, best_generation, best_generation_time, times = genetic_algorithm_deap(
            param_ranges=param_ranges,
            population_size=hyperparams["population_size"],
            generations=20,
            mutation_rate=hyperparams["mutation_rate"],
            crossover_rate=hyperparams["crossover_rate"],
            elitism_fraction=hyperparams["elitism_fraction"],
            alpha=hyperparams["alpha"],
            mu=hyperparams["mu"],
            sigma=hyperparams["sigma"],
            tournsize=hyperparams["tournsize"],
            crossover_method=tools.cxBlend,
            mutation_method=tools.mutGaussian,
            selection_method=tools.selTournament
        )

        run_data = {
            'Best Fitness': best_fitness,
            'Best Generation': best_generation,
            'Best Generation Time': best_generation_time,
            'Iteration Times': times
        }
        results.append(run_data)

    df_results = pd.DataFrame(results)
    median_fitness = df_results['Best Fitness'].median()
    mode_iteration = df_results['Best Generation'].mode()[0]
    median_time_to_best_gen = pd.Series([min(r['Iteration Times']) for r in results]).median()

    summary_row = {
        'Медиана MSE': median_fitness,
        'Мода лучшей итерации': mode_iteration,
        'Медиана времени до лучшей итерации': median_time_to_best_gen,
        'Population Size': hyperparams["population_size"],
        'Mutation Rate': hyperparams["mutation_rate"],
        'Crossover Rate': hyperparams["crossover_rate"],
        'Elitism Fraction': hyperparams["elitism_fraction"],
        'Alpha': hyperparams["alpha"],
        'Mu': hyperparams["mu"],
        'Sigma': hyperparams["sigma"],
        'Tournament Size': hyperparams["tournsize"]
    }

    summary_results.append(summary_row)

    # Сохранение в CSV после каждого внешнего цикла
    df_summary = pd.DataFrame([summary_row])
    save_results_to_csv(df_summary, filename)

    gc.collect()

# Итоговая таблица с результатами всех внешних прогонов
df_summary = pd.DataFrame(summary_results)
print("\nИтоговая таблица с медианными значениями и гиперпараметрами:")
print(df_summary)


Внешние прогоны:   0%|          | 0/2 [00:00<?, ?it/s]


=== Внешний прогон 1 ===

Поколение 1, Лучшее MSE: 512301.7168504563, Параметры: [233.05579352712908, 11.301928446636651, 478.05469759075964, 0.036699661298565216, 417.35453860513456, 0.8060237397169621]
Поколение 2, Лучшее MSE: 512301.7168504563, Параметры: [233.05579352712908, 11.301928446636651, 478.05469759075964, 0.036699661298565216, 417.35453860513456, 0.8060237397169621]
Поколение 3, Лучшее MSE: 512301.7168504563, Параметры: [233.05579352712908, 11.301928446636651, 478.05469759075964, 0.036699661298565216, 417.35453860513456, 0.8060237397169621]
Поколение 4, Лучшее MSE: 509482.87433572445, Параметры: [1532.3044811805494, 16.0716379699398, 620.1419266549842, 0.01, 77.85808982978622, 0.7959485584950627]
Поколение 5, Лучшее MSE: 509482.87433572445, Параметры: [1532.3044811805494, 16.0716379699398, 620.1419266549842, 0.01, 77.85808982978622, 0.7959485584950627]
Поколение 6, Лучшее MSE: 509482.87433572445, Параметры: [1532.3044811805494, 16.0716379699398, 620.1419266549842, 0.01, 7

c:\Users\oftuadmin\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\oftuadmin\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Поколение 1, Лучшее MSE: 507561.41341153276, Параметры: [364.8830213034472, 52.48587998533016, 202.20622034014238, 0.029516450169005552, 210.44530416536702, 0.7786895548150987]
Поколение 2, Лучшее MSE: 507561.41341153276, Параметры: [364.8830213034472, 52.48587998533016, 202.20622034014238, 0.029516450169005552, 210.44530416536702, 0.7786895548150987]
Поколение 3, Лучшее MSE: 507561.41341153276, Параметры: [364.8830213034472, 52.48587998533016, 202.20622034014238, 0.029516450169005552, 210.44530416536702, 0.7786895548150987]
Поколение 4, Лучшее MSE: 507561.41341153276, Параметры: [364.8830213034472, 52.48587998533016, 202.20622034014238, 0.029516450169005552, 210.44530416536702, 0.7786895548150987]
Поколение 5, Лучшее MSE: 507561.41341153276, Параметры: [364.8830213034472, 52.48587998533016, 202.20622034014238, 0.029516450169005552, 210.44530416536702, 0.7786895548150987]
Поколение 6, Лучшее MSE: 507561.41341153276, Параметры: [364.8830213034472, 52.48587998533016, 202.20622034014238, 

c:\Users\oftuadmin\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\oftuadmin\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


...

Поколение 1, Лучшее MSE: 514643.8278890853, Параметры: [1530.1625387092024, 32.0244592434665, 885.9083546129951, 0.01, 269.2329582431444, 0.6543112795373106]
Поколение 2, Лучшее MSE: 511527.48914700217, Параметры: [708.041911919292, 20.39633701749822, 516.5503105227172, 0.018339991755710577, 819.7256753491811, 0.7351716039704174]
Поколение 3, Лучшее MSE: 511527.48914700217, Параметры: [708.041911919292, 20.39633701749822, 516.5503105227172, 0.018339991755710577, 819.7256753491811, 0.7351716039704174]
Поколение 4, Лучшее MSE: 509702.3701320489, Параметры: [671.573974639327, 12.421368772081564, 556.2236139864381, 0.016809556678051053, 568.3727248267232, 0.7884552337638844]
Поколение 5, Лучшее MSE: 509702.3701320489, Параметры: [671.573974639327, 12.421368772081564, 556.2236139864381, 0.016809556678051053, 568.3727248267232, 0.7884552337638844]
Поколение 6, Лучшее MSE: 509702.3701320489, Параметры: [671.573974639327, 12.421368772081564, 556.2236139864381, 0.016809556678051053, 568.3727248

c:\Users\oftuadmin\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\oftuadmin\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Поколение 1, Лучшее MSE: 511879.5243291629, Параметры: [875.5369255628503, 16.47661275330976, 811.4895074841116, 0.01, 924.2752860414124, 0.9663552482424411]
Поколение 2, Лучшее MSE: 507773.61420393037, Параметры: [1182.0322123253352, 85.7736146548446, 822.57996909817, 0.01, 637.726421159774, 0.724016299368334]
Поколение 3, Лучшее MSE: 507773.61420393037, Параметры: [1182.0322123253352, 85.7736146548446, 822.57996909817, 0.01, 637.726421159774, 0.724016299368334]
Поколение 4, Лучшее MSE: 507773.61420393037, Параметры: [1182.0322123253352, 85.7736146548446, 822.57996909817, 0.01, 637.726421159774, 0.724016299368334]
Поколение 5, Лучшее MSE: 507607.47788855684, Параметры: [1187.1857707794998, 80.8216475710017, 777.2161839937539, 0.010000000000000002, 816.9395011541703, 0.7238970436537778]
Поколение 6, Лучшее MSE: 507607.47788855684, Параметры: [1187.1857707794998, 80.8216475710017, 777.2161839937539, 0.010000000000000002, 816.9395011541703, 0.7238970436537778]
Поколение 7, Лучшее MSE: 50

Внешние прогоны: 100%|██████████| 2/2 [2:54:44<00:00, 5242.25s/it]

Поколение 20, Лучшее MSE: 507464.6345497556, Параметры: [1190.4848659818663, 94.6894711742944, 756.2384910032002, 0.010000000000000002, 713.3574573217132, 0.7258383053722257]

Итоговая таблица с медианными значениями и гиперпараметрами:
     Медиана MSE  Мода лучшей итерации  Медиана времени до лучшей итерации   
0  507561.413412                     1                           11.589091  \
1  507662.643368                    18                           31.331649   

   Population Size  Mutation Rate  Crossover Rate  Elitism Fraction     Alpha   
0               48       0.222773        0.103229          0.116557  0.250839  \
1               56       0.251391        0.477170          0.097989  0.532701   

         Mu     Sigma  Tournament Size  
0 -0.981995  0.183521                2  
1  0.864021  0.741242                3  
